Install or import the libaries and modules

In [ ]:
# Install libraries.
import numpy as np
import pandas as pd
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import my_functions as func
import get_my_data as getData


# Create a Dash app to interact with the plots.

Create an interactive Dash app that will show the plots and parameters of interest.

In [ ]:
# Initialize the Dash App
app = dash.Dash(__name__)

# Define the Layout
app.layout = html.Div([
    dcc.Dropdown(
        id='year-selector',
        options=[{'label': year, 'value': year} for year in getData.dataframes.keys()],
        value='2013',  # Default value
        style={'width': '50%'}
    ),
    dcc.Slider(
        id='vmin-slider',
        min=np.log10(0.0001),  # log10 of actual min
        max=np.log10(1),  # log10 of actual max
        step=0.01,  # Step in log10 scale
        value=np.log10(0.0001),  # log10 of default value
        marks={np.log10(x): str(x) for x in [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1]}  # Logarithmic marks
    ),
    html.Div([
        dcc.Graph(id='data-plot'),
        dcc.Graph(id='cdf-plot')
    ], style={'display': 'flex', 'justify-content': 'space-between'}),
    html.Div(id='result-display', style={'color': 'white', 'backgroundColor': 'black'})
])

# Define the Callback to Update the Plots
@app.callback(
    [Output('data-plot', 'figure'),
     Output('cdf-plot', 'figure'),
     Output('result-display', 'children')],
    [Input('year-selector', 'value'),
     Input('vmin-slider', 'value')]
)
def update_plots(selected_year, log_selected_vmin):
    # Get the selected dataframe
    df = getData.dataframes[selected_year]
    number_of_days = getData.year_to_days[selected_year]

    # Convert log-selected vmin back to linear scale
    selected_vmin = 10 ** log_selected_vmin

    # Get the volumes greater than the selected minimum.
    volumes_truncated = df.Volume[df.Volume >= selected_vmin]
    n_truncated = len(volumes_truncated)
    
    # Calculate the empirical cumulative distribution function.
    ecdf = func.ecdf(n_truncated)

    # Calculate the theoretical cumulative distribution function.
    tcdf, b_hat = func.tcdf(volumes_truncated, selected_vmin, n_truncated)

    # Calculate the weighted Kolmogorov-Smirnov statistics.
    D_star, p_value = stats.ks_2samp(ecdf, tcdf)

    # Calculate the power law fit values.
    v_max = df['Volume'].max()
    x_values, y_annual = func.power_law(v_max, selected_vmin, n_truncated, number_of_days, b_hat)

    #Plot normalized rank vs. volumes on the left plot.
    fig1 = px.scatter(df, x='Volume', y='Normalized Rank', log_x=True, log_y=True)

    # Add a trace for the selected vmin line to the left plot.
    fig1.add_trace(go.Scatter(
        x=[selected_vmin, selected_vmin],
        y=[df['Normalized Rank'].min(), df['Normalized Rank'].max()],
        mode='lines',
        name='selected mnimimum volume'
    ))

    # Add the power curve to fit the truncated volume range to left plot.
    fig1.add_trace(go.Scatter(x=x_values, y=y_annual,
                                 mode='lines',
                                 name=f'Power-law fit {selected_year}',
                                 line=dict(color='black', width=1)))

    # Create the right plot with the two CDFs.
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=volumes_truncated, y=ecdf, mode='lines', name='ECDF'))
    fig2.add_trace(go.Scatter(x=volumes_truncated, y=tcdf, mode='lines', name='TCDF'))
    fig2.update_xaxes(type="log")

    # Print the current KS statistics.
    result_text = f"v_min: {selected_vmin}, D*: {D_star}, p-value: {p_value}, b: {b_hat}, Number of events: {n_truncated}"

    return fig1, fig2, result_text

# Run the app
if __name__ == '__main__':
  app.run_server(mode='external')
